In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
import re

In [2]:
data = pd.read_csv('Sentiment.csv')

# Keeping only the neccessary columns
data = data[['text','sentiment']]


In [3]:
def preProcess_data(text):
   text = text.lower()
   new_text = re.sub('[^a-zA-z0-9\s]','',text)
   new_text = re.sub('rt', '', new_text)
   return new_text

data['text'] = data['text'].apply(preProcess_data)


In [4]:
max_fatures = 2000

tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X, 28) 

Y = pd.get_dummies(data['sentiment']).values

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20)

In [6]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.3, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(128,recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])


In [7]:
batch_size = 512

model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, validation_data=(X_test, Y_test))


Epoch 1/10
22/22 [==============================] - 24s 905ms/step - loss: 0.9606 - accuracy: 0.5967 - val_loss: 0.8940 - val_accuracy: 0.6054
Epoch 2/10
22/22 [==============================] - 20s 917ms/step - loss: 0.8408 - accuracy: 0.6247 - val_loss: 0.8083 - val_accuracy: 0.6486
Epoch 3/10
22/22 [==============================] - 21s 938ms/step - loss: 0.7441 - accuracy: 0.6718 - val_loss: 0.7675 - val_accuracy: 0.6703
Epoch 4/10
22/22 [==============================] - 20s 925ms/step - loss: 0.6869 - accuracy: 0.6995 - val_loss: 0.7571 - val_accuracy: 0.6735
Epoch 5/10
22/22 [==============================] - 21s 945ms/step - loss: 0.6440 - accuracy: 0.7226 - val_loss: 0.7455 - val_accuracy: 0.6746
Epoch 6/10
22/22 [==============================] - 21s 937ms/step - loss: 0.6091 - accuracy: 0.7349 - val_loss: 0.7585 - val_accuracy: 0.6793
Epoch 7/10
22/22 [==============================] - 21s 936ms/step - loss: 0.5885 - accuracy: 0.7431 - val_loss: 0.7660 - val_accuracy: 0.6829

In [8]:
model.save('sentiment.h5')